In [1]:
import pandas as pd
import ast
import spacy
from spacy.pipeline import EntityRuler
from sklearn.metrics import precision_recall_fscore_support

In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
data = pd.read_csv("PET_Customized.csv")

In [4]:
def extract_tasks(sentence):
    doc = nlp(sentence)
    tasks = []

    for token in doc:
        if token.dep_ == "ROOT": 
            action = token.lemma_
            actor, obj = None, None

            for child in token.children:
                if child.dep_ in ("nsubj", "nsubjpass"):  
                    actor = child.text
                elif child.dep_ in ("obl:agent", "pobj"): 
                    actor = child.text
                elif child.dep_ in ("dobj", "attr"): 
                    obj = child.text

            if actor is None:
                actor = "System"  

            tasks.append({"Actor": actor, "Action": action, "Object": obj})

    return tasks

In [5]:
data["extracted_tasks"] = data["original"].apply(extract_tasks)

data = data.explode("extracted_tasks")

data[["Actor", "Action", "Object"]] = data["extracted_tasks"].apply(pd.Series)

data.drop(columns=["extracted_tasks"], inplace=True)

In [6]:
data["tokens"] = data["tokens"].apply(eval)  
data["ner_tags"] = data["ner_tags"].apply(eval)  

In [7]:
task_label_mapping = {
    "B-Actor": "Actor",
    "I-Actor": "Actor",
    "B-Activity": "Action",
    "I-Activity": "Action",
    "B-Activity Data": "Object",
    "I-Activity Data": "Object"
}

In [8]:
def extract_gold_entities(tokens, ner_tags):
    entities = {"Actor": [], "Action": [], "Object": []}
    current_entity = None
    current_tokens = []

    for token, tag in zip(tokens, ner_tags):
        if tag.startswith("B-"):  
            if current_entity: 
                entities[current_entity].append(" ".join(current_tokens))
            current_entity = task_label_mapping.get(tag, None)
            current_tokens = [token] if current_entity else []
        elif tag.startswith("I-") and current_entity:  # Inside an entity
            current_tokens.append(token)
        else: 
            if current_entity:
                entities[current_entity].append(" ".join(current_tokens))
            current_entity = None
            current_tokens = []

    if current_entity:
        entities[current_entity].append(" ".join(current_tokens))

    return entities

In [9]:
def evaluate_extraction(row):
    tokens = row["tokens"]
    gold_labels = row["ner_tags"]

    gold_entities = extract_gold_entities(tokens, gold_labels)

    predicted_tasks = extract_tasks(" ".join(tokens))  

    predicted_labels = ["O"] * len(tokens)

    for task in predicted_tasks:
        for i, token in enumerate(tokens):
            if task["Actor"] and token in task["Actor"].split():
                predicted_labels[i] = "B-Actor" if i == 0 else "I-Actor"
            elif task["Action"] and token in task["Action"].split():
                predicted_labels[i] = "B-Activity" if i == 0 else "I-Activity"
            elif task["Object"] and token in task["Object"].split():
                predicted_labels[i] = "B-Activity Data" if i == 0 else "I-Activity Data"

    return predicted_labels, gold_labels

In [10]:
data["predicted_ner"], data["gold_ner"] = zip(*data.apply(evaluate_extraction, axis=1))

all_predicted = [label for sublist in data["predicted_ner"] for label in sublist]
all_gold = [label for sublist in data["gold_ner"] for label in sublist]

precision, recall, f1, _ = precision_recall_fscore_support(all_gold, all_predicted, average="weighted")

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

Precision: 0.3768
Recall: 0.4838
F1-score: 0.3608


C:\Users\hello\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
